In [2]:
import pickle
import numpy as np

In [3]:
with open("train_qa.txt", "rb") as fp:   # Unpickling, read-binary
    train_data =  pickle.load(fp)

In [4]:
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

In [5]:
print(type(test_data),type(train_data),len(test_data),len(train_data))


<class 'list'> <class 'list'> 1000 10000


In [6]:
train_data[1005]

(['Daniel',
  'journeyed',
  'to',
  'the',
  'hallway',
  '.',
  'John',
  'picked',
  'up',
  'the',
  'apple',
  'there',
  '.'],
 ['Is', 'Daniel', 'in', 'the', 'hallway', '?'],
 'yes')

In [7]:
text=''
print('Story:')
for sent in train_data[99]:
    if sent!='yes' and sent!='no':
        for word in sent:
            if (word!='.'):
                if (word!='?'):
                    text+= word + ' '
                else:
                    print()
                    print('Question:', text[:-1]+word)
                    print()
            else:
                print(text[:-1]+word)
                text=''
    else:
        print('Answer:', sent)

Story:
Daniel grabbed the apple there.
Daniel went to the bedroom.
John moved to the garden.
Sandra journeyed to the office.
Daniel put down the apple.
Mary went to the bedroom.
Mary grabbed the apple there.
Sandra went back to the garden.
Mary went to the kitchen.
Daniel went to the office.

Question: Is Mary in the garden?

Answer: no


In [8]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [9]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [10]:
train_data[0][2]

'no'

In [11]:
# Create a set that holds the vocab words
vocab = set()

In [12]:
all_data = test_data + train_data

In [13]:
len(all_data)

11000

In [14]:
set(train_data[0][0])

{'.',
 'Mary',
 'Sandra',
 'bathroom',
 'bedroom',
 'journeyed',
 'moved',
 'the',
 'to'}

In [15]:
for story, question , answer in all_data:
    # In case you don't know what a union of sets is:
    # https://www.programiz.com/python-programming/methods/set/union
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

Putting the answer Possibilities:

In [16]:
vocab.add('no')
vocab.add('yes')

In [17]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [18]:
len(vocab)

37

In [19]:
len_vocab= len(vocab)+1 #0 for Keras's pad_sequences
len_vocab

38

In [20]:
max_story_len = max([len(data[0]) for data in all_data])
max_story_len

156

In [21]:
max_question_len = max([len(data[1]) for data in all_data])
max_question_len

6

### VECTORIZING THE DATA

In [22]:
# Reserve 0 for pad_sequences
vocab_size = len(vocab) + 1

In [23]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [24]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])   # provide empty list for filter out
tokenizer.fit_on_texts(vocab)

In [25]:
tokenizer.word_index

{'journeyed': 1,
 'bathroom': 2,
 'garden': 3,
 'went': 4,
 'got': 5,
 'up': 6,
 'football': 7,
 'hallway': 8,
 'put': 9,
 'in': 10,
 'john': 11,
 'travelled': 12,
 'back': 13,
 'milk': 14,
 'dropped': 15,
 'there': 16,
 'apple': 17,
 'left': 18,
 'the': 19,
 'kitchen': 20,
 'yes': 21,
 'discarded': 22,
 'down': 23,
 'sandra': 24,
 '?': 25,
 'bedroom': 26,
 'took': 27,
 'picked': 28,
 'mary': 29,
 'grabbed': 30,
 'daniel': 31,
 'office': 32,
 'no': 33,
 'to': 34,
 'is': 35,
 'moved': 36,
 '.': 37}

In [26]:

train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [27]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [28]:
len(train_story_text)

10000

# Function for Vectorizing

In [29]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(vocab_size)  # this includes +1 for padding
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [30]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [31]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [32]:
inputs_test

array([[ 0,  0,  0, ..., 19, 26, 37],
       [ 0,  0,  0, ..., 19,  3, 37],
       [ 0,  0,  0, ..., 19,  3, 37],
       ...,
       [ 0,  0,  0, ..., 19, 17, 37],
       [ 0,  0,  0, ..., 19,  3, 37],
       [ 0,  0,  0, ..., 17, 16, 37]], dtype=int32)

In [33]:
queries_test

array([[35, 11, 10, 19, 20, 25],
       [35, 11, 10, 19, 20, 25],
       [35, 11, 10, 19,  3, 25],
       ...,
       [35, 29, 10, 19, 26, 25],
       [35, 24, 10, 19,  3, 25],
       [35, 29, 10, 19,  3, 25]], dtype=int32)

In [34]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
       503.,   0.,   0.,   0.,   0.])

In [36]:
tokenizer.word_index['yes']

21

In [37]:
tokenizer.word_index['no']

33

# Creating the Model

In [38]:

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input, Activation, Dense, Permute, Dropout
from tensorflow.keras.layers import add, dot, concatenate
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop

There are 2 inputs: stories and questions. We use placeholders

In [39]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

# Network Building

Input Encoder M

In [40]:
embedding_dim = 128

# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim= embedding_dim))
input_encoder_m.add(Dropout(0.3))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

Input Encoder C

In [41]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

Question Encoder

In [42]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=embedding_dim,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

Encode the Sequence

In [43]:
# encode input sequence and questions (which are indices) to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

Use dot product to compute the match between first input vector seq and the query

In [44]:
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

Add this match matrix with the second input vector sequence

In [45]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

Concatenate

In [46]:
# concatenate the match matrix with the question vector sequence
# (samples, query_maxlen, story_maxlen + embedding_dim)
answer = concatenate([response, question_encoded])

In [47]:
answer

<KerasTensor: shape=(None, 6, 284) dtype=float32 (created by layer 'concatenate')>

In [48]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # shape (samples, 32)

In [49]:
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

In [50]:
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
optimizer = RMSprop(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 156)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 sequential (Sequential)     (None, None, 128)            4864      ['input_1[0][0]']             
                                                                                                  
 sequential_2 (Sequential)   (None, 6, 128)               4864      ['input_2[0][0]']             
                                                                                              

In [52]:
import math
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

initial_learning_rate = 0.01
epochs = 120
decay = initial_learning_rate / epochs

def lr_step_decay(epoch, lr):
    drop_rate = 0.5
    epochs_drop = 20
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

learning_rate = LearningRateScheduler(lr_step_decay, verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.66, patience=5, min_lr=0.0001, verbose=1)  # factor by which the learning rate will be reduced. new_lr = lr * factor

In [53]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=256,epochs=120,validation_data=([inputs_test, queries_test], answers_test))  # , callbacks=[reduce_lr]


Epoch 1/120
40/40 [==============================] - 3s 42ms/step - loss: 0.9041 - accuracy: 0.4869 - val_loss: 0.7415 - val_accuracy: 0.4970
Epoch 2/120
40/40 [==============================] - 1s 33ms/step - loss: 0.7099 - accuracy: 0.4955 - val_loss: 0.6939 - val_accuracy: 0.4970
Epoch 3/120
40/40 [==============================] - 2s 38ms/step - loss: 0.7027 - accuracy: 0.4951 - val_loss: 0.7512 - val_accuracy: 0.4970
Epoch 4/120
40/40 [==============================] - 2s 42ms/step - loss: 0.7015 - accuracy: 0.5089 - val_loss: 0.6941 - val_accuracy: 0.4970
Epoch 5/120
40/40 [==============================] - 2s 40ms/step - loss: 0.7011 - accuracy: 0.5051 - val_loss: 0.6954 - val_accuracy: 0.5030
Epoch 6/120
40/40 [==============================] - 2s 40ms/step - loss: 0.7029 - accuracy: 0.4936 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 7/120
40/40 [==============================] - 1s 36ms/step - loss: 0.7013 - accuracy: 0.4978 - val_loss: 0.7165 - val_accuracy: 0.4970
Epoch 

40/40 [==============================] - 1s 37ms/step - loss: 0.4959 - accuracy: 0.7695 - val_loss: 0.4924 - val_accuracy: 0.7580
Epoch 59/120
40/40 [==============================] - 1s 36ms/step - loss: 0.4669 - accuracy: 0.7891 - val_loss: 0.5068 - val_accuracy: 0.7500
Epoch 60/120
40/40 [==============================] - 1s 37ms/step - loss: 0.4455 - accuracy: 0.7940 - val_loss: 0.5040 - val_accuracy: 0.7580
Epoch 61/120
40/40 [==============================] - 1s 36ms/step - loss: 0.4332 - accuracy: 0.8020 - val_loss: 0.4689 - val_accuracy: 0.7850
Epoch 62/120
40/40 [==============================] - 1s 36ms/step - loss: 0.4183 - accuracy: 0.8106 - val_loss: 0.4976 - val_accuracy: 0.7690
Epoch 63/120
40/40 [==============================] - 2s 45ms/step - loss: 0.4123 - accuracy: 0.8104 - val_loss: 0.4309 - val_accuracy: 0.8140
Epoch 64/120
40/40 [==============================] - 1s 36ms/step - loss: 0.4020 - accuracy: 0.8206 - val_loss: 0.4363 - val_accuracy: 0.8030
Epoch 65/120

40/40 [==============================] - 1s 36ms/step - loss: 0.2650 - accuracy: 0.8834 - val_loss: 0.4993 - val_accuracy: 0.8090
Epoch 116/120
40/40 [==============================] - 1s 36ms/step - loss: 0.2620 - accuracy: 0.8817 - val_loss: 0.4381 - val_accuracy: 0.8190
Epoch 117/120
40/40 [==============================] - 1s 36ms/step - loss: 0.2592 - accuracy: 0.8854 - val_loss: 0.5194 - val_accuracy: 0.8130
Epoch 118/120
40/40 [==============================] - 1s 36ms/step - loss: 0.2578 - accuracy: 0.8891 - val_loss: 0.4678 - val_accuracy: 0.8190
Epoch 119/120
40/40 [==============================] - 1s 36ms/step - loss: 0.2541 - accuracy: 0.8911 - val_loss: 0.4958 - val_accuracy: 0.8200
Epoch 120/120
40/40 [==============================] - 1s 37ms/step - loss: 0.2574 - accuracy: 0.8889 - val_loss: 0.5523 - val_accuracy: 0.8080


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('accuracy.png', dpi=180, facecolor='white')
plt.show()

NameError: name 'model' is not defined

In [54]:
#model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

32/32 [==============================] - 0s 2ms/step


In [56]:
test_data[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [57]:
story =' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [58]:
query = ' '.join(word for word in test_data[0][1])
print(query)

Is John in the kitchen ?


In [59]:
print("Real answer:",test_data[0][2])

True Test Answer from Data is: no


In [60]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer:", k)
print("Accuracy: ", 100*pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.9999926
